# 1. What is the fundamental idea behind Support Vector Machines?

## My Solution 

Support Vector Machines attempt to minimize the instances within a margin for classification purposes. The larger the margin, the better we are at predicting if something is x or y. Instances on the margin are called support vectors. 

Likewise, it can be extended to regression. In this we maximize the instances within a margin. 

# 2. What is a support vector?

## My Solution 

A support vector is an instance on the margin line, or "side walk" of our street.



# 3. Why is it important to scale the inputs when using SVMs?

## My solution 

SVMs do not used a closed form equation for minmizing the objective function. More so, the model the margins are sensetive to large magnitueds. Due to this, scaling allows us to maximize our margin and allows us to quickly optimize our cost function. 

# 4. Can an SVM classifier output a confidence score when it classifies an instance? What about a probability?

## My solution
An SVM classifier does not output a probability as with a logicisitic model. 


# 5. Should you use the primal or the dual form of the SVM problem to train a model on a training set with millions of instances and hundreds of features?

## My solution

If we do not wish to increase model complexity, it is best to use an optimized classifier. Else we should use the dual form of the SVM problem. The dual problem allows us to easily find our optimized parameters, while leaving the floor open to kernels. 

# 6. Say you’ve trained an SVM classifier with an RBF kernel, but it seems to underfit the training set. Should you increase or decrease γ (gamma)? What about C?

## My  Solution 

If our model is underfitting, we should increase gamma. $C$ follows the same logic.

# 7. How should you set the QP parameters (H, f, A, and b) to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?






## My solution 

This was not a question I can immediatly answer, looking at the notes: 

If we also want to avoid any margin violations (hard margin), then we need the decision function to be greater than 1 for all positive training instances and lower than –1 for negative training instances. If we define t(i) = –1 for negative instances (if y(i) = 0) and t(i) = 1 for positive instances (if y(i) = 1), then we can express this constraint as t(i)(w⊺ x(i) + b) ≥ 1 for all instances.

Soft margins can have instances within the margins, because of this our decision function should allow instances in $[-1,1]$. Thus we should change our matrix A.I do not know what exactly to change. 


## Book Solution 



# 8. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model.

In [1]:
# From my understanding, the iris dataset was linearly seperable. 
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

In [2]:
#Training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= .1,  random_state=32)

In [3]:
len(X_train) # Should be quick enough to train

135

# Linear SVM

In [4]:
from sklearn.svm import SVC
linear_svc = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ])
linear_svc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [5]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(linear_svc, X_train, y_train, cv=3)

In [6]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train, y_train_pred)

0.9347826086956522

In [7]:
recall_score(y_train, y_train_pred)

0.9347826086956522

So far this model is killing it. 

## SVC with a Guassian Kernel

In [8]:
from sklearn.svm import SVC
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [9]:
y_train_pred = cross_val_predict(rbf_kernel_svm_clf, X_train, y_train, cv=3)
precision_score(y_train, y_train_pred, zero_division=1)

1.0

In [10]:
#Having some issues with the recall/ precision. Looking at accuracy another way. 
recall_score(y_train, y_train_pred, zero_division=1)

0.0

In [11]:
y_pred = rbf_kernel_svm_clf.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7333333333333333

Our model needs some improvement. We can use gridsearch to optimize the hyperparameters

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'C': [.25 , .5, 1,2], 'gamma': [.25, .5, 1,2, 3]},
  ]

svc = SVC(kernel="rbf")
grid_search = GridSearchCV(svc, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

In [14]:
svc.get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [15]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.25, 0.5, 1, 2],
                          'gamma': [0.25, 0.5, 1, 2, 3]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [16]:
grid_search.best_params_

{'C': 0.25, 'gamma': 2}

In [17]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=2, C=0.25))
    ])
rbf_kernel_svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.25, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=2,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [18]:
y_pred = rbf_kernel_svm_clf.predict(X_test)
accuracy_score(y_test, y_pred) #this was what the linearsvc was at 

0.9333333333333333

## SGDClassifier

In [19]:
from sklearn.linear_model import SGDClassifier
m = len(X_train)
C =1
SGD_class = SGDClassifier(loss="hinge", alpha=1/(m*C))

In [20]:
SGD_class.fit(X_train, y_train)

SGDClassifier(alpha=0.007407407407407408, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
#OverFitts the data? It works well that's for sure! 
y_pred = SGD_class.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

# 9. 

Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?



In [22]:
#Fetches MNIST Dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [36]:
X, y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

- Note : with a linear SVM classifier. It will automatically use the One-vs-All (also called One-vs-the-Rest, OvR) strategy, so there's nothing special we need to do. Easy!

S/O github solutions. I originally thought that I had to split the data into 10 categories for each numeric. 

In [24]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [38]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [39]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9703703703703703

This linear model is certainly too simple for MNIST, but perhaps we just needed to scale the data first:

In [40]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [42]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred) #some how made it worse? 

0.9555555555555556

we want to use an SVM, we will have to use a kernel. Let's try an SVC with an RBF kernel (the default).

In [44]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9629629629629629

In [45]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=9.716094651077585, gamma=0.008487313201851318 .................
[CV] .. C=9.716094651077585, gamma=0.008487313201851318, total=   0.0s
[CV] C=9.716094651077585, gamma=0.008487313201851318 .................
[CV] .. C=9.716094651077585, gamma=0.008487313201851318, total=   0.0s
[CV] C=9.716094651077585, gamma=0.008487313201851318 .................
[CV] .. C=9.716094651077585, gamma=0.008487313201851318, total=   0.0s
[CV] C=5.871892669890743, gamma=0.0037301057716828726 ................
[CV] . C=5.871892669890743, gamma=0.0037301057716828726, total=   0.0s
[CV] C=5.871892669890743, gamma=0.0037301057716828726 ................
[CV] . C=5.871892669890743, gamma=0.0037301057716828726, total=   0.0s
[CV] C=5.871892669890743, gamma=0.0037301057716828726 ................
[CV] . C=5.871892669890743, gamma=0.0037301057716828726, total=   0.0s
[CV] C=6.355427480332562, gamma=0.0031094140544452803 ................
[CV] . C=6.35542

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A61CC7ECF8>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A61CC7EF98>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [46]:
rnd_search_cv.best_estimator_

SVC(C=9.716094651077585, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3,
    gamma=0.008487313201851318, kernel='rbf', max_iter=-1, probability=False,
    random_state=None, shrinking=True, tol=0.001, verbose=False)

In [47]:

rnd_search_cv.best_score_

0.9629629629629629

In [48]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)
#I'm surprised this ran quickly on my rig 

0.9555555555555556

In [49]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9333333333333333

For some off reason the linear model fit best. K-cross folds validiton might be useful to verfiy the performance of the model. I gave up after a while and this is what the book did. 

# 10. 
Train an SVM regressor on the California housing dataset.

In [50]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=42, tol=0.0001, verbose=0)

In [54]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9612806653297273

In [55]:
np.sqrt(mse) #RMSE

0.9804492160890983

- Note: Recall that RMSE is almost the equivalent of one standard deviation. Scaling the data means the standard deviation is in units of whatever we are measuring(our label). Thus an RMSE of almost one indicates we have an standard deviation of 1 unit. In this data set we are measuring homes in the cost of 10s of thousands of dollars.

In this training set, the targets are tens of thousands of dollars. The RMSE gives a rough idea of the kind of error you should expect (with a higher weight for large errors): so with this model we can expect errors somewhere around $10,000. Not great. Let's see if we can do better with an RBF Kernel. We will use randomized search with cross validation to find the appropriate hyperparameter values for C and gamma:

In [56]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   8.9s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   8.8s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................
[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   8.9s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=  10.3s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=   8.3s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=   8.9s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=   6.7s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=   7.8s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='scale', kernel='rbf',
                                 max_iter=-1, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A61CC808D0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A61CC800F0>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [57]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.07969454818643928, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [58]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785357

In [59]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.5929168385528746

Our standard deviation is 60% of one unit. About 6000 per standard deviation. 